In [1]:
# library use
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')
from sklearn import preprocessing


from sklearn.metrics import mean_squared_error

import pickle
%config InlineBackend.figure_format = 'retina'

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import sys
import pandas as pd

In [2]:
# metric 구성

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score

class CustomMetric():

    def __init__(self, pred, real):
            self.pred = pred
            self.real = real
            self.thr25 = np.percentile(self.real, 25)
            self.thr75 = np.percentile(self.real, 75)
            self.df = pd.DataFrame({'pred' : self.pred, 'real' : self.real})
        
    def issame(self, data1, data2):
        if data1 > 0 and data2 > 0 : 
            return 'T'
        elif data1 > 0 and data2 < 0 :
            return 'F'
        elif data1 < 0 and data2 > 0 :
            return 'F'
        elif data1 < 0 and data2 < 0 :
            return 'T'
        elif data1 == 0 or data2 == 0 :
            return 0
        else :
            return 'notcal'

    def getouter(self, data1, data2): #quantile_25 = np.percentile(values, 25)

        if data1 > 0 and data2 >= self.thr75 :
            return 'T'
        elif data1 < 0 and data2 <= self.thr25 :
            return 'T'
        elif data1 >= 0 and data2 <= self.thr25 :
            return 'F'
        elif data1 <= 0 and data2 >= self.thr75: 
            return 'F'
        else:
            return 'notcal'
        
    def makedf(self):
        self.df['TF'] = self.df.apply(lambda x : self.issame(x['pred'], x['real']), axis = 1)
        self.df['thrTF'] = self.df.apply(lambda x : self.getouter(x['pred'], x['real']), axis = 1)
        
        return self.df
        
    def execution(self):
        mdf = pd.DataFrame()
        mdf['CORR'] = [self.df['real'].corr(self.df['pred'], method = 'pearson')]
        mdf['R2'] = ["{0:0.4f}".format(r2_score(self.df['real'], self.df['pred']))]
        mdf['MAE'] = ["{0:0.4f}".format(mean_absolute_error(self.df['pred'], self.df['real']))]
        mdf['RMSE'] = ["{0:0.4f}".format(np.sqrt(mean_squared_error(self.df['pred'], self.df['real'])))]
        mdf['ACR'] = [sum(self.df['TF'] == 'T')/len(self.df['TF'])]
        
        mdf['threshACR'] = [sum(self.df['thrTF'] == 'T') / sum(self.df['thrTF'] != 'notcal')]
        
        return mdf        
    

In [29]:
data = []
with open('return_result_1month.dat', 'r') as f:
    lines = f.readlines()
    for i in lines:        
        data.append(i.rstrip("\n").split('|'))

In [30]:
columns = data[0]
datadf = data[1:]

In [31]:
df = pd.DataFrame(datadf, columns = columns, dtype = float)

In [32]:
df.head()

,DATE,Actual Value,Prediction Value
0,20161108.0,-0.000485,0.000283
1,20161109.0,0.000072,0.000196
2,20161110.0,0.001318,0.000930
3,20161111.0,0.001293,-0.000092
4,20161114.0,0.001739,0.000734


In [7]:
df.DATE = pd.to_datetime(df.DATE).astype(int)

In [8]:
df.DATE = df.DATE.astype(str)

In [9]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')
df['YEAR'] = df['DATE'].apply(lambda x: x.year)

In [10]:
df.corr()

,Actual Value,Prediction Value,YEAR
Actual Value,1.000000,0.260127,-0.040190
Prediction Value,0.260127,1.000000,0.066641
YEAR,-0.040190,0.066641,1.000000


In [11]:
year2016.corr()

NameError: name 'year2016' is not defined

In [12]:
year2017.corr()

NameError: name 'year2017' is not defined

In [13]:
df['Actual Value'].corr(df['Prediction Value'], method = 'pearson')

0.26012696279647823

In [14]:
year2016['Actual Value'].corr(year2016['Prediction Value'], method = 'pearson')

NameError: name 'year2016' is not defined

In [15]:
year2017['Actual Value'].corr(year2017['Prediction Value'], method = 'pearson')

NameError: name 'year2017' is not defined

In [16]:
year2018['Actual Value'].corr(year2018['Prediction Value'], method = 'pearson')

NameError: name 'year2018' is not defined

In [17]:
df.tail()

,DATE,Actual Value,Prediction Value,YEAR
704,2019-09-24,0.004760,0.001192,2019
705,2019-09-25,-0.012853,0.001379,2019
706,2019-09-26,0.001711,0.001166,2019
707,2019-09-27,-0.013035,0.000158,2019
708,2019-09-30,0.008149,0.001386,2019


In [18]:
col = list(df['YEAR'].unique())

In [19]:
df.set_index('YEAR', inplace = True)

In [20]:
mod = sys.modules[__name__]

In [21]:
for i, j in enumerate(col):
    setattr(mod, 'year'+ str(col[i]), df.loc[col[i]])

In [22]:
metr = CustomMetric(df['Prediction Value'], df['Actual Value'])
metr2016 = CustomMetric(year2016['Prediction Value'], year2016['Actual Value'])
metr2017 = CustomMetric(year2017['Prediction Value'], year2017['Actual Value'])
metr2018 = CustomMetric(year2018['Prediction Value'], year2018['Actual Value'])
#metr2019 = CustomMetric(year2019['Prediction Value'], year2019['Actual Value'])

In [23]:
metr.makedf()
metr2016.makedf()
metr2017.makedf()
metr2018.makedf()
#metr2019.makedf()

,pred,real,TF,thrTF
YEAR,,,,
2018,-0.001228,0.003873,F,notcal
2018,0.002423,0.003460,T,notcal
2018,0.002770,-0.007826,F,F
2018,0.000242,0.013435,T,T
2018,0.000931,0.006997,T,T
...,...,...,...,...
2018,-0.004461,0.000944,F,notcal
2018,-0.003602,-0.002304,T,notcal
2018,-0.004215,-0.012825,T,T


In [24]:
metr.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,0.260127,0.0649,0.0059,0.0080,0.596615,0.643258


In [25]:
metr2016.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,0.495627,0.2124,0.0049,0.0071,0.605263,0.7


In [26]:
metr2017.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,0.264183,0.0622,0.0047,0.0063,0.621399,0.680328


In [27]:
metr2018.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,0.311816,0.0855,0.0068,0.0090,0.610656,0.672131


In [28]:
metr2019.execution()

NameError: name 'metr2019' is not defined